In [579]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from scipy.stats import skewnorm
from scipy.stats import norm

from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from textwrap import wrap

from mdutils.mdutils import MdUtils
from mdutils import Html
import networkx as nx
import random
import pyodbc
import math
import sys

from abc import ABC, abstractmethod

from networkx.drawing.nx_agraph import write_dot, graphviz_layout

import warnings

warnings.filterwarnings('ignore')

In [2]:
def Dice(x,y,**kwargs):
    z = kwargs.get('drop_num')
    high = kwargs.get('drop_high')
    low = kwargs.get('drop_low')
    dice = y+1
    bottom = kwargs.get('bottom',1)
    roll_rng = range(bottom,dice)
    roll = [*roll_rng]
    if bool(z):
        roll.remove(z)
    # print(roll)
    result = []
    for i in range(1,x+1):
        res = np.random.choice(roll)
        result.append(res)
    # print(result)
    if bool(high):
        result.sort(reverse=False)
        result.pop()
    # print(result)
    if bool(low):
        result.sort(reverse=True)
        result.pop()
    # print(result)
    Sum = sum(result)
    return Sum


def setall(d, keys, value):
    for k in keys:
        d[k] = value

In [3]:
class Hex:
    # Initializing constructor
    def __init__(self, **kwargs):
        self.Terrain = kwargs.get('Ter')
        self.Climate = kwargs.get('Clim')
        self.Location = kwargs.get('Loc')
        self.Num_Features = kwargs.get('NF',Dice(1,6))
        self.Num_Lairs = kwargs.get('NL',Dice(1,6))
 
    # def isMammal(self):
    #     if self.mammals:
    #         print("It is a mammal.")
 
    # def isIntelligent(self):
    #     if self.intelligent:
    #         print("It is an intelligent creature.")

    # def name(self,f,l,m):
    #     self.first = f
    #     self.last = l
    #     self.mid = m
    #     self.full = (self.first,self.mid,self.last)

class Lair:
    def __init__(self, **kwargs):
        self.Number = kwargs.get('Num')


In [660]:
Biome_Codes = {
(0,0.125):  ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(0,0.25):   ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(0,0.5):    ['Polar/Alvar Desert', 'Glacier', 'Barren Rock','Frozen Earth','Ice Sheet'],
(1,0.125):  ['Rain Tundra',],
(1,0.25):   ['Wet Tundra',],
(1,0.5):    ['Moist Tundra',],
(1,1):      ['Dry Tundra','Mammoth Steppe'],
(2,0.125):  ['Boreal Rain Forest','Boreal Bog','Boreal Fen'],
(2,0.25):   ['Wet Boreal Forest','Taiga','Mammoth Steppe'],
(2,0.5):    ['Moist Boreal Forest','Fungal Forest','Taiga','Mammoth Steppe'],
(2,1):      ['Dry Scrub','Fern Prairie','Mammoth Steppe'],
(2,2):      ['Desert','Fog Desert'],
(3,0.125):  ['Rain Forest','Moorland','Cloud Forest'],
(3,0.25):   ['Wet Forest','Prototaxite Forest','Moorland','Cloud Forest'],
(3,0.5):    ['Moist Forest','Fungal Forest','Moorland','Cloud Forest'],
(3,1):      ['Steppe','Moorland'],
(3,2):      ['Desert Scrub',],
(3,4):      ['Desert','Fog Desert'],
(4,0.125):  ['Rain Forest',],
(4,0.25):   ['Wet Forest',],
(4,0.5):    ['Moist Forest',],
(4,1):      ['Dry Forest',],
(4,2):      ['Thorn Steppe',],
(4,4):      ['Desert Scrub',],
(4,8):      ['Desert',],
(5,0.125):  ['Tropical Rain Forest',],
(5,0.25):   ['Wet Forest',],
(5,0.5):    ['Moist Forest','Peat Swamp Forest'],
(5,1):      ['Dry Forest',],
(5,2):      ['Very Dry Forest',],
(5,4):      ['Thorn Woodland',],
(5,8):      ['Desert Scrub',],
(5,16):     ['Desert',    ],
}

Water_Table = {
    0:'Dry',
    1:'Confined-Aquifer',
    2:'Unconfined-Aquifer',
    3:'Confined ',
    4:'Ground-Saturating',
    5:'Ponds',
    6:'Wetlands',
    7:'Surface-Water',
}

absorbance_index = {
    7    :0.8125,
    6    :0.825 ,
    5    :0.8375,
    4    :0.85  ,
    3    :0.8625,
    2    :0.875 ,
    1    :0.8875,
    0    :0.9   ,
}

# Hazardous_Atmosphere_Composition = {
#     0:'High-Halogen', ##Highly toxic to organisms with earth-like biochemistry; bio-organic polymers in this environment would be more like plastics, air would be colored, distort images, pool in low-lying areas, rain is acidic
#     1:'High-Inert', ##risk inert gas narcosis in dense or greater atmospheres
#     2:'High-C02', ##Hyperventilation and easy fatigue
#     3:'High-O2', ##Fire Hazard and easy Fatigue
#     4:'Low-O2', ##Reduce density effect on breathing by 1 level, may make it easier to breathe in dense atmospheres, but will cause easy fatigue in thin atmospheres
# }

humidity_code = {
    0.125:'Super-Humid',
    0.25:'Per-Humid',
    0.5:'Humid',
    1:'Sub-Humid',
    2:'Semi-Arid',
    4:'Arid',
    8:'Perarid',
    16:'Super-Arid',
}

humidity_index = {
    0.125:0.185 ,
    0.25 :0.18 ,
    0.5  :0.175 ,
    1    :0.17 ,
    2    :0.165 ,
    4    :0.16 ,
    8    :0.1575 ,
    16   :0.15 ,
}

Precipitation_Probability = {
    'Super-Humid':90,
    'Per-Humid':80,
    'Humid':65,
    'Sub-Humid':32,
    'Semi-Arid':16,
    'Arid':8,
    'Perarid':4,
    'Super-Arid':2,
}

Altitude_belts = [
    ("Alvar",0),
    ("Alpine",1),
    ("Subalpine",2),
    ("Montane",3),
   ( "Premontane",4),
   ( "Sea-Level",5),
   ("Lowlands",6),
]


Average_Atmosphere_Density = [
    0.8,
    0.85,
    0.9,
    0.95,
    1,
    1.05,
    1.1,
    1.15,
    1.2,
]

# Atmosphere_Dict = {
#     0.8:'Trace',
#     0.85:'Sparse',
#     0.9:'Thin',
#     0.95:'Sub-Standard',
#     1:'Standard',
#     1.05:'Above-Average',
#     1.1:'Dense',
#     1.15:'Very-Dense',
#     1.2:'Super-Dense',
# }

Alt_and_atmos = {
    "Alvar":0.5,
    "Alpine":0.75,
    "Subalpine":1,
    "Montane":1.25,
    "Premontane":1.5,
    "Sea-Level":1.75,
    "Lowlands":2,
}

Latitude_regions = [
    ("Polar",0),
    ("Subpolar",1),
    ("Boreal",2),
    ("Temperate",3),
    ("Subtropical",4),
    ("Tropical",5),
    ("Equatorial",6),
]

# temp_belts = {
#     0:'Frozen',
#     1:'Cold',
#     2:'Cool',
#     3:'Mild',
#     4:'Warm',
#     5:'Hot',
#     6:'Scorching',
# }

Avg_Temp_kelvin = {
    0:250,
    1:265,
    2:280,
    3:295,
    4:310,
    5:325,
    6:340,
}

groundwater_type = [
    "freshwater",
    "salt water",
    "brackish"
]

Ocean_Dist_Bands = {
    0:'Ocean', ## This hex is ocean water
    1:'Small Island', ## This hex is surrounded on all sides by Ocean hexes
    2:'Beach', ## At least one side of this hex borders the ocean, and at least one side does not
    3:'Coastal', ## At least one side of this of this hex borders a beach hex, but it does not border any ocean hexes
    4:'Inland', ## At least one side of this hex borders a coastal hex, but it does not border any beach or ocean hexes
    5:'Mainland', ##This hex only borders Mainland or Inland hexes; it cannot border any other hex-type from this list
}

Ocean_Effect = {
    0:-20, ### Effect of distance from ocean on Temperature Std. Dev.
    1:-15,
    2:-10,
    3:-5,
    4:0,
    5:10,
}







class Biome:
    def __init__(self, **kwargs):
        self.Altitude = kwargs.get('Alt',random.choice(Altitude_belts))
        self.Latitude = kwargs.get('Alt',random.choice(Latitude_regions))
        self.biotemp = min(self.Altitude[1],self.Latitude[1])
        Evapotrans_ratios = [2**n for n in range(-3,self.biotemp)]
        self.Evap = kwargs.get('Evap',random.choice(Evapotrans_ratios))
        self.Humid = kwargs.get('Humid',humidity_code[self.Evap])
        self.code = (self.biotemp,self.Evap)
        # self.Biome = kwargs.get('biome',Biome_Codes[self.code])
        # self.Temp_belt = kwargs.get('temp',temp_belts[self.biotemp])

        self.water_level = kwargs.get('Water',random.choice(list(Water_Table.keys())))
        self.Ocean_Dist = kwargs.get('OD',random.choice(list(Ocean_Dist_Bands.keys())))
        ocean_eff = Ocean_Effect[self.Ocean_Dist]
        self.avg_atmos = kwargs.get('atmos',random.choice(Average_Atmosphere_Density))
        # self.atmos_type = Atmosphere_Dict[self.avg_atmos]
        self.relative_atmos = Alt_and_atmos[self.Altitude[0]]*self.avg_atmos
        self.blackbody = absorbance_index[self.water_level]*(1+(self.relative_atmos*humidity_index[self.Evap]))
        temp = math.floor(1.8*(Avg_Temp_kelvin[self.biotemp]/self.blackbody) - 460 + random.choice([-30,-20,-10,0,10,20,30,40]))
        temp_stdev = abs(abs(math.floor(np.random.normal(loc=0.0, scale=20.0, size=None)))+ocean_eff)
        self.Temp =(temp,temp_stdev)



In [677]:
B = Biome()
print(B.Altitude[0])
print(B.Latitude[0])
# print(B.Temp_belt)
print(B.Humid)
# print(B.Biome)
# print(B.atmos_type)
print(B.Temp)

Alpine
Tropical
Super-Humid
(86, 11)


In [ ]:
class soil:
    def __init__(self, **kwargs):
        self.temp = kwargs.get('temp')
        self.evap = kwargs.get('evap')

In [11]:
Flora_Density = [
    'High',
    'Medium',
    'Low',
    'Barren'
]

Fauna_Density = [
    'High',
    'Medium',
    'Low',
    'Barren'
]

Soil = [
    "Volcanic Glass",
    "Volcanic Soil",
    "Sand",
    "Sandy Soil",
    "Clay",
    "Loam",
    "Silt",
    "Peat",
    "Permafrost",
    "Terra-Rossa",
    

]
Rocks = [
    "Boulders",
    "Large rocks",
    "Small Rocks",
    "Pebbles",
    "Gravel",
    "None"
]

groundwater_type = [
    "freshwater",
    "salt water",
    "brackish"
]
underground_aquifer = [
    "yes",
    "no"
]

Unusual_terrain = [
    "Glacier",
    "Hot Springs",
    "Landlocked Sand Dunes",
    "Mudflats",
    "Petrified Forest",
    "Rare Rock Formations",
    "Salt Flat",
    "Volcano",
    "Craters",
    
]
